In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
from unsloth import FastLanguageModel
import torch

# 모델 설정 파라미터
max_seq_length = 4096 # 원하는 길이로 설정! 내부적으로 RoPE 스케일링을 자동 지원합니다!
dtype = None # 자동 감지용. Tesla T4, V100는 Float16, Ampere+는 Bfloat16 사용
load_in_4bit = True # 메모리 사용량을 줄이기 위해 4비트 양자화 사용. False로 설정 가능

# 4비트 사전 양자화된 모델 목록 - 4배 빠른 다운로드와 메모리 부족 현상 방지
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2배 더 빠름
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 405B 모델용 4비트!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22B 2배 더 빠름!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2배 더 빠름!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2배 더 빠름!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # 신규! Llama 3.2 모델
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # 신규! Llama 3.3 70B!
] # 더 많은 모델은 https://huggingface.co/unsloth 에서 확인 가능

# 모델 및 토크나이저 로딩
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # 또는 "unsloth/Llama-3.2-1B-Instruct" 선택 가능
    max_seq_length = max_seq_length,
    dtype = dtype,
    #load_in_4bit = load_in_4bit,
    load_in_4bit = False,  # ⚠️ 이게 핵심!
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [3]:
# PEFT(Parameter-Efficient Fine-Tuning) 모델 설정
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # 0보다 큰 숫자 선택! 추천값: 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # 모든 값 지원, 0이 최적화됨
    bias = "none",    # 모든 값 지원, "none"이 최적화됨
    # [신규] "unsloth"는 VRAM 사용량을 30% 줄이고 배치 크기를 2배로 늘립니다!
    use_gradient_checkpointing = "unsloth", # 매우 긴 컨텍스트의 경우 True 또는 "unsloth" 사용
    random_state = 3407,
    use_rslora = False,  # rank stabilized LoRA 지원
    loftq_config = None, # LoftQ 지원
)


Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:

# 데이터셋 로딩 및 전처리
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

# 데이터셋 구조 확인을 위한 디버깅 코드 추가
print("데이터셋 구조 확인 중...")
dataset = load_dataset("mlabonne/FineTome-100k", split="train")
print("첫 번째 샘플의 키:", dataset[0].keys())
print("데이터셋 구조:", dataset)

# 3. 각 데이터셋을 ShareGPT 포맷으로 정제
def convert_finetome(example):
    # 데이터셋 구조에 맞게 수정
    conversations = example.get("conversations", [])
    if not conversations:
        conversations = [
            {"from": "user", "value": example.get("instruction", example.get("input", ""))},
            {"from": "assistant", "value": example.get("output", example.get("response", ""))},
        ]
    return {"conversations": conversations}

def convert_webgpt(example):
    """WebGPT 데이터셋을 ShareGPT 형식으로 변환"""
    try:
        # 질문 처리
        question = example.get("question", {})
        if isinstance(question, dict):
            question_text = question.get("full_text", "")
        else:
            question_text = str(question)

        # 인용문 처리
        quotes_0 = example.get("quotes_0", {})
        quotes_text = ""
        if isinstance(quotes_0, dict):
            titles = quotes_0.get("title", [])
            extracts = quotes_0.get("extract", [])
            if titles and extracts:
                quotes_text = "\n".join([
                    f"Title: {title}\nExtract: {extract}"
                    for title, extract in zip(titles, extracts)
                ])

        # 질문과 인용문 결합
        full_question = f"{question_text}"
        if quotes_text:
            full_question += f"\n\n[Web Results]\n{quotes_text}"

        # 답변 처리
        answer = example.get("answer_0", "")
        if not answer:
            answer = example.get("answer_1", "")

        # 빈 대화 필터링
        if not full_question.strip() and not answer.strip():
            return None

        return {
            "conversations": [
                {"from": "user", "value": full_question},
                {"from": "assistant", "value": answer}
            ]
        }
    except Exception as e:
        print(f"변환 중 오류 발생: {e}")
        print("예시 데이터:", example)
        return None

def convert_longalign(example):
    # LongAlign-10k 데이터셋 구조에 맞게 수정
    conversations = example.get("conversations", [])
    if not conversations:
        conversations = [
            {"from": "user", "value": example.get("instruction", example.get("input", ""))},
            {"from": "assistant", "value": example.get("output", example.get("response", ""))},
        ]
    return {"conversations": conversations}

def convert_longbench(example):
    # LongBench-v2 데이터셋 구조에 맞게 수정
    question = example.get("question", "")
    context = example.get("context", "")
    if context:
        question = f"{context}\n\n{question}"

    answer = example.get("answer", "")
    if not answer and "choice" in example:
        # 객관식 답변 처리
        choices = [v for k, v in example.items() if k.startswith("choice_")]
        answer = "\n".join(f"{chr(65+i)}. {choice}" for i, choice in enumerate(choices))
        answer += f"\n정답: {example.get('answer', '')}"

    return {
        "conversations": [
            {"from": "user", "value": question},
            {"from": "assistant", "value": answer},
        ]
    }

# 4. 데이터셋 로딩 및 변환
print("데이터셋 로딩 및 변환 시작...")
try:
    ds1 = load_dataset("mlabonne/FineTome-100k", split="train").map(convert_finetome)
    print("FineTome 데이터셋 변환 완료")
except Exception as e:
    print(f"FineTome 데이터셋 변환 중 오류 발생: {e}")
    print("데이터셋 구조를 확인하세요.")
    raise

try:
    # WebGPT 데이터셋 구조 확인
    print("\nWebGPT 데이터셋 구조 확인 중...")
    webgpt_dataset = load_dataset("openai/webgpt_comparisons", split="train")
    print("첫 번째 샘플의 키:", webgpt_dataset[0].keys())
    print("첫 번째 샘플:", webgpt_dataset[0])

    ds2 = webgpt_dataset.map(convert_webgpt)
    print("WebGPT 데이터셋 변환 완료")
except Exception as e:
    print(f"WebGPT 데이터셋 변환 중 오류 발생: {e}")
    print("데이터셋 구조:", webgpt_dataset)
    ds2 = None

try:
    # LongAlign-10k 데이터셋 로딩
    ds3 = load_dataset("THUDM/LongAlign-10k", split="train").map(convert_longalign)
    print("LongAlign-10k 데이터셋 변환 완료")
except Exception as e:
    print(f"LongAlign-10k 데이터셋 변환 중 오류 발생: {e}")
    ds3 = None

try:
    # LongBench-v2 데이터셋 로딩
    ds4 = load_dataset("THUDM/LongBench-v2", split="train").map(convert_longbench)
    print("LongBench-v2 데이터셋 변환 완료")
except Exception as e:
    print(f"LongBench-v2 데이터셋 변환 중 오류 발생: {e}")
    ds4 = None

# 데이터셋 병합
from datasets import concatenate_datasets
try:
    datasets_to_merge = [ds1]
    if ds2 is not None:
        datasets_to_merge.append(ds2)
    if ds3 is not None:
        datasets_to_merge.append(ds3)
    if ds4 is not None:
        datasets_to_merge.append(ds4)

    # 각 데이터셋의 구조를 통일
    for ds in datasets_to_merge:
        ds = ds.map(lambda x: {"conversations": x["conversations"]})

    dataset = concatenate_datasets(datasets_to_merge)
    print("데이터셋 병합 완료")
    print(f"병합된 데이터셋 크기: {len(dataset)}")
except Exception as e:
    print(f"데이터셋 병합 중 오류 발생: {e}")
    dataset = ds1


데이터셋 구조 확인 중...
첫 번째 샘플의 키: dict_keys(['conversations', 'source', 'score'])
데이터셋 구조: Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})
데이터셋 로딩 및 변환 시작...
FineTome 데이터셋 변환 완료

WebGPT 데이터셋 구조 확인 중...


0000.parquet:   0%|          | 0.00/127M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19578 [00:00<?, ? examples/s]

첫 번째 샘플의 키: dict_keys(['question', 'quotes_0', 'answer_0', 'tokens_0', 'score_0', 'quotes_1', 'answer_1', 'tokens_1', 'score_1'])
첫 번째 샘플: {'question': {'dataset': 'triviaqa', 'id': '18c654a169eb80287f4353d33e701b1c', 'full_text': 'Voiced by Harry Shearer, what Simpsons character was modeled after Ted Koppel?'}, 'quotes_0': {'title': ['Kent Brockman (en.wikipedia.org)', 'Krusty the Clown (en.wikipedia.org)'], 'extract': ['Kent Brockman is a fictional character in the animated television series The Simpsons. He is voiced by Harry Shearer and first appeared in the episode "Krusty Gets Busted". He is a grumpy, self-centered local Springfield news anchor.', "Krusty was created by cartoonist Matt Groening and partially inspired by Rusty Nails, a television clown from Groening's hometown of Portland, Oregon."]}, 'answer_0': 'The Simpsons character that was possibly based on Ted Koppel is Kent Brockman.  He is a local news anchor in Springfield and is modeled after Ted Koppel. [1]', 'tokens_0

Map:   0%|          | 0/19578 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/19578 [00:00<?, ? examples/s]

Map:   0%|          | 0/9888 [00:00<?, ? examples/s]

Map:   0%|          | 0/503 [00:00<?, ? examples/s]

데이터셋 병합 중 오류 발생: The features can't be aligned because the key length of features {'_id': Value(dtype='string', id=None), 'domain': Value(dtype='string', id=None), 'sub_domain': Value(dtype='string', id=None), 'difficulty': Value(dtype='string', id=None), 'length': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'choice_A': Value(dtype='string', id=None), 'choice_B': Value(dtype='string', id=None), 'choice_C': Value(dtype='string', id=None), 'choice_D': Value(dtype='string', id=None), 'answer': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'conversations': [{'from': Value(dtype='string', id=None), 'value': Value(dtype='string', id=None)}]} has unexpected type - Value(dtype='string', id=None) (expected either Value(dtype='int64', id=None) or Value("null").


In [6]:

# 5. ShareGPT 스타일 표준화
print("ShareGPT 스타일 표준화 시작...")
try:
    # 데이터셋 구조 확인
    print("데이터셋 샘플 확인:", dataset[0])

    # 빈 대화 필터링
    dataset = dataset.filter(lambda x: any(msg["value"].strip() for msg in x["conversations"]))

    dataset = standardize_sharegpt(dataset)
    print("ShareGPT 스타일 표준화 완료")
except Exception as e:
    print(f"ShareGPT 스타일 표준화 중 오류 발생: {e}")
    raise

ShareGPT 스타일 표준화 시작...
데이터셋 샘플 확인: {'conversations': [{'from': 'human', 'value': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programmi

Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=12):   0%|          | 0/100000 [00:00<?, ? examples/s]

ShareGPT 스타일 표준화 완료


In [7]:
# 6. text 필드 생성 (chat template 적용)
def formatting_prompts_func(examples):
    try:
        texts = [
            tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
            for convo in examples["conversations"]
        ]
        return { "text": texts }
    except Exception as e:
        print(f"프롬프트 포맷팅 중 오류 발생: {e}")
        print("예시 데이터:", examples)
        raise

print("프롬프트 포맷팅 시작...")
dataset = dataset.map(formatting_prompts_func, batched=True)
print("데이터셋 전처리 완료")

프롬프트 포맷팅 시작...


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

데이터셋 전처리 완료


In [8]:
# 학습 설정
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # 짧은 시퀀스의 경우 학습 속도를 5배까지 향상시킬 수 있음
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # 전체 학습을 위해 이 값을 설정
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # WandB 등에 사용
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [9]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

# 입력과 레이블 디코딩 확인
tokenizer.decode(trainer.train_dataset[5]["input_ids"])
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

# @title 현재 GPU 메모리 상태 출력
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"사용 중인 GPU: {gpu_stats.name}, 최대 메모리: {max_memory} GB")
print(f"학습 시작 시점에 예약된 메모리: {start_gpu_memory} GB")

# 학습 실행
trainer_stats = trainer.train()

# @title 최종 GPU 메모리 사용량 및 학습 시간 출력
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

print(f"총 학습 시간: {trainer_stats.metrics['train_runtime']} 초")
print(f"총 학습 시간(분 단위): {round(trainer_stats.metrics['train_runtime']/60, 2)} 분")
print(f"최대 예약 메모리 사용량: {used_memory} GB")
print(f"LoRA 학습에 사용된 추가 메모리: {used_memory_for_lora} GB")
print(f"전체 GPU 메모리 대비 최대 예약 메모리 비율: {used_percentage} %")
print(f"전체 GPU 메모리 대비 LoRA 학습 메모리 비율: {lora_percentage} %")


Map (num_proc=12):   0%|          | 0/100000 [00:00<?, ? examples/s]

사용 중인 GPU: NVIDIA A100-SXM4-40GB, 최대 메모리: 39.557 GB
학습 시작 시점에 예약된 메모리: 6.779 GB


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,237,063,680 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.769700
2,0.825900
3,1.079100
4,0.883500
5,0.759900
6,0.933300
7,0.615500
8,0.988200
9,0.849200
10,0.749400


총 학습 시간: 171.5723 초
총 학습 시간(분 단위): 2.86 분
최대 예약 메모리 사용량: 7.426 GB
LoRA 학습에 사용된 추가 메모리: 0.647 GB
전체 GPU 메모리 대비 최대 예약 메모리 비율: 18.773 %
전체 GPU 메모리 대비 LoRA 학습 메모리 비율: 1.636 %


In [10]:

from google.colab import drive, files
# 3. 저장 디렉토리 세팅
drive.mount('/content/drive', force_remount=True)

drive_dir = "/content/drive/MyDrive/unsloth_models/Llama_3.2_3B_test/"
os.makedirs(drive_dir, exist_ok=True)

from peft import PeftModel

print(type(model))
# 또는
print(model.__class__)

# 4. 모델 저장
# base_model = model.base_model
# base_model.save_pretrained(f"{drive_dir}/base_model_saved")
# tokenizer.save_pretrained(f"{drive_dir}/base_model_saved")
model = model.merge_and_unload()

model.save_pretrained(f"{drive_dir}/lora_model")
tokenizer.save_pretrained(f"{drive_dir}/lora_model")

model.save_pretrained_merged(f"{drive_dir}/merged_16bit", tokenizer, save_method="merged_16bit")
# model.save_pretrained_merged(f"{drive_dir}/merged_4bit", tokenizer, save_method="merged_4bit_forced")

# model.save_pretrained_gguf(f"{drive_dir}/gguf_f16", tokenizer, quantization_method="f16")
# model.save_pretrained_gguf(f"{drive_dir}/gguf_q4_k_m", tokenizer, quantization_method="q4_k_m")

# GGUF 모델은 vLLM이나 HuggingFace `transformers`에서 직접 로드 불가.
# GGUF용 추론은 llama.cpp 또는 koboldcpp에서 처리 필요 (ex. llama-cpp-python). 하단 참조.


Mounted at /content/drive
<class 'peft.peft_model.PeftModelForCausalLM'>
<class 'peft.peft_model.PeftModelForCausalLM'>


Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.4G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 50.44 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:00<00:00, 691.00it/s]

Unsloth: Saving tokenizer...

 Done.
Done.


In [2]:
!pip install -U "vllm[triton]" accelerate
!pip install -U peft transformersw
# vLLM 0.8.4는 내부적으로 zmq.asyncio.Context()를 사용해
# 멀티 프로세스(MP) 서버를 띄우는데,
# Colab이나 특정 로컬 환경에서 zmq의 버전 또는 context 초기화 방식이 꼬일 수 있습니다.

# ✅ 해결 방법
# 🔧 1. pyzmq 버전 업/다운그레이드
!pip install pyzmq==25.1.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 125.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.

ERROR: Could not find a version that satisfies the requirement transformersw (from versions: none)
ERROR: No matching distribution found for transformersw
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.8 MB/s eta 0:00:00
  Attempting uninstall: pyzmq
    Found existing installation: pyzmq 24.0.1
    Uninstalling pyzmq-24.0.1:
      Successfully uninstalled pyzmq-24.0.1


In [8]:
drive_dir = "/content/drive/MyDrive/unsloth_models/Llama_3.2_3B_test/"
!python3 -m vllm.entrypoints.openai.api_server \
  --model {drive_dir}lora_model \
  --dtype float16 \
  --port 8000 \
  --gpu-memory-utilization 0.6 \
  --max-model-len 4096

INFO 04-18 07:29:35 [__init__.py:239] Automatically detected platform cuda.
2025-04-18 07:29:36.123397: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-18 07:29:36.139514: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744961376.160248    6682 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744961376.166603    6682 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-18 07:29:36.186808: I tensorflow/core/platform/cpu_feature_

In [ ]:
# !python3 -m vllm.entrypoints.openai.api_server \
#   --model {drive_dir}merged_16bit \
#   --dtype float16 \
#   --port 8000 \
#   --gpu-memory-utilization 0.9 \
#   --max-model-len 4096

In [ ]:
# !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
# !chmod +x cloudflared

# # cloudflared 터널 시작
# !./cloudflared tunnel --url http://localhost:8000

In [7]:
# import time, re

# from google.colab import drive, files
# # 3. 저장 디렉토리 세팅
# drive.mount('/content/drive', force_remount=True)
# drive_dir = "/content/drive/MyDrive/unsloth_models/Llama_3.2_3B_test/"
# # 1. vLLM 서버 백그라운드 실행
# !python3 -m vllm.entrypoints.openai.api_server \
#   --model {drive_dir}merged_16bit \
#   --dtype float16 \
#   --port 8000 \
#   --gpu-memory-utilization 0.6 \
#   --max-model-len 4096 > vllm.log 2>&1 &
#   # --api-key YOUR_SECRET_KEY \

# # !python3 -m vllm.entrypoints.openai.api_server \
# #   --model {drive_dir}lora_model \
# #   --dtype float16 \
# #   --port 8000 \
# #   --gpu-memory-utilization 0.9 \
# #  # --api-key YOUR_SECRET_KEY \
# #   --max-model-len 4096 > vllm.log 2>&1 &

# # 2. cloudflared 다운로드 및 복사
# !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
# !cp cloudflared cloudflared_exec
# !chmod +x cloudflared_exec

# # 3. cloudflared 백그라운드 실행
# !./cloudflared_exec tunnel --url http://localhost:8000 --no-autoupdate > cloudflared.log 2>&1 &

# # 4. Cloudflared URL 확인
# time.sleep(15)
# with open("cloudflared.log") as f:
#     log = f.read()

# match = re.search("https://.*trycloudflare.com", log)
# public_url = match.group(0) if match else None

# print("🔗 외부에서 접근 가능한 URL:")
# print(public_url if public_url else "❌ URL 찾기 실패")
# !ps -ef | grep cloudflared
# !tail -n 50 cloudflared.log

import time
# import re # ngrok 사용 시 정규식 파싱 불필요
from google.colab import drive, files
!pip install pyngrok -q # ngrok 파이썬 라이브러리 설치
from pyngrok import ngrok

# 1. Google Drive 마운트 및 저장 디렉토리 설정
print("☁️ Google Drive 마운트 중...")
drive.mount('/content/drive', force_remount=True)
drive_dir = "/content/drive/MyDrive/unsloth_models/Llama_3.2_3B_test/"
print(f"✅ Google Drive 마운트 완료. 모델 디렉토리: {drive_dir}")

# 2. vLLM 서버 백그라운드 실행 (기존과 동일)
print("🚀 vLLM 서버를 백그라운드에서 시작합니다...")
!python3 -m vllm.entrypoints.openai.api_server \
  --model {drive_dir}merged_16bit \
  --dtype float16 \
  --port 8000 \
  --gpu-memory-utilization 0.6 \
  --max-model-len 4096 > vllm.log 2>&1 &
  # --api-key YOUR_SECRET_KEY \

# 주석 처리된 다른 모델 로딩 옵션 (필요시 주석 해제)
# !python3 -m vllm.entrypoints.openai.api_server \
#   --model {drive_dir}lora_model \
#   --dtype float16 \
#   --port 8000 \
#   --gpu-memory-utilization 0.9 \
#  # --api-key YOUR_SECRET_KEY \
#   --max-model-len 4096 > vllm.log 2>&1 &

# vLLM 서버가 포트를 열고 준비될 시간을 잠시 기다립니다.
print("⏳ vLLM 서버 시작 대기 중 (약 15-30초 소요될 수 있음)...")
time.sleep(20) # 서버 시작 시간을 고려하여 충분히 대기 (필요시 시간 조절)

# 3. ngrok 설정 (Authtoken 설정 - 선택 사항이지만 권장)
# ngrok 계정이 있고 Authtoken을 사용하려면 아래 주석을 해제하고 'YOUR_NGROK_AUTHTOKEN' 부분을 실제 토큰으로 바꾸세요.
# 안정적인 연결과 추가 기능을 위해 Authtoken 사용을 권장합니다.
# 웹사이트: https://dashboard.ngrok.com/get-started/your-authtoken
print("🔑 ngrok Authtoken 설정 시도...")
try:
    ngrok.set_auth_token("2vtLqb9HoP0xoiiwqovFN2laGrd_6pD67sTGUxgGasqEDTwHb") # <-- 여기에 ngrok Authtoken을 입력하세요
    print("✅ ngrok Authtoken 설정 완료.")
except Exception as e:
    print(f"⚠️ ngrok Authtoken 설정 중 오류 발생 (무시하고 진행): {e}")

# 4. ngrok 터널 시작 및 URL 확인
public_url = None
print("🚇 ngrok 터널을 시작합니다 (로컬 포트 8000)...")
try:
    # localhost의 8000번 포트로 http 터널을 생성합니다.
    tunnel = ngrok.connect(8000, "http")
    public_url = tunnel.public_url
    print("✅ ngrok 터널 생성 완료!")
    print(f"🔗 외부에서 접근 가능한 URL: {public_url}")

    # 5. ngrok 프로세스 및 터널 정보 확인 (선택 사항)
    print("\n🔍 실행 중인 ngrok 프로세스 확인:")
    !ps -ef | grep ngrok

    print("\n📊 활성 ngrok 터널 정보:")
    # get_tunnels()는 활성화된 터널 객체 리스트를 반환합니다.
    active_tunnels = ngrok.get_tunnels()
    if active_tunnels:
        for t in active_tunnels:
            print(t)
    else:
        print("활성 ngrok 터널이 없습니다.")

except Exception as e:
    print(f"❌ ngrok 터널 생성 실패: {e}")
    print("🤔 vLLM 서버 로그 또는 ngrok 관련 설정을 확인하세요.")
    # 실패 시 vLLM 로그의 마지막 부분을 출력하여 디버깅에 도움을 줄 수 있습니다.
    print("\n📄 vLLM 로그 마지막 20줄:")
    !tail -n 20 vllm.log

# 주의: 이 Colab 셀이 실행 중인 동안에만 ngrok 터널이 유지됩니다.
# 만약 터널을 계속 유지해야 한다면, 이 셀이 종료되지 않도록 하거나
# 추가적인 코드를 마지막에 넣어둘 수 있습니다 (예: input("엔터키를 눌러 종료..."))
if public_url:
  print(f"\n✅ vLLM 서버가 {public_url} 에서 실행 중입니다.")
  print("   Colab 셀 실행이 중지되면 터널도 함께 종료됩니다.")
else:
  print("\n❌ 터널 URL을 가져오지 못했습니다.")

☁️ Google Drive 마운트 중...
Mounted at /content/drive
✅ Google Drive 마운트 완료. 모델 디렉토리: /content/drive/MyDrive/unsloth_models/Llama_3.2_3B_test/
🚀 vLLM 서버를 백그라운드에서 시작합니다...
⏳ vLLM 서버 시작 대기 중 (약 15-30초 소요될 수 있음)...
🔑 ngrok Authtoken 설정 시도...
✅ ngrok Authtoken 설정 완료.
🚇 ngrok 터널을 시작합니다 (로컬 포트 8000)...
✅ ngrok 터널 생성 완료!
🔗 외부에서 접근 가능한 URL: https://f853-35-240-172-74.ngrok-free.app

🔍 실행 중인 ngrok 프로세스 확인:
root        6446     989 16 07:29 ?        00:00:00 /root/.config/ngrok/ngrok start --none --log=stdout
root        6469     989  0 07:29 ?        00:00:00 /bin/bash -c ps -ef | grep ngrok
root        6471    6469  0 07:29 ?        00:00:00 grep ngrok

📊 활성 ngrok 터널 정보:
NgrokTunnel: "https://f853-35-240-172-74.ngrok-free.app" -> "http://localhost:8000"

✅ vLLM 서버가 https://f853-35-240-172-74.ngrok-free.app 에서 실행 중입니다.
   Colab 셀 실행이 중지되면 터널도 함께 종료됩니다.


|목적|	추천|
|:-|:-|
|고속 API 서빙 (Chat, Search 등)	|✅ vLLM|
|브라우저, 노트북, IoT Edge 추론	|✅ GGUF|
|프롬프트 길거나 병렬 추론 많음	|✅ vLLM 필수|
|단일 사용자, 간단한 테스트/디버깅	|✅ GGUF 충분|

In [7]:
#GGUF 서빙은 의미 없음.
!lsof -i:8000